# Week 10 Discussion & Q&A:
### Parallelization for Hardware Accelerators (e.g., GPUs) 


In [1]:
import Pkg
#Pkg.activate("../../lab8-start/")
Pkg.activate(".")

  Activating project at `/storage/work/ebf11/Teach/Astro528/Fall2023/Notes-Fall2023/week10`


In [2]:
using CUDA, Test

In [3]:
CUDA.versioninfo()

CUDA runtime 12.2, artifact installation
CUDA driver 12.2
NVIDIA driver 535.104.12

CUDA libraries: 
- CUBLAS: 12.2.5
- CURAND: 10.3.3
- CUFFT: 11.0.8
- CUSOLVER: 11.5.2
- CUSPARSE: 12.1.2
- CUPTI: 20.0.0
- NVML: 12.0.0+535.104.12

Julia packages: 
- CUDA: 5.0.0
- CUDA_Driver_jll: 0.6.0+4
- CUDA_Runtime_jll: 0.9.2+3

Toolchain:
- Julia: 1.9.2
- LLVM: 14.0.6
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5
- Device capability support: sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: Tesla P100-PCIE-12GB (sm_60, 11.502 GiB / 12.000 GiB available)


# P100 GPU layout
![P100 Block diagram](https://cdn.arstechnica.net/wp-content/uploads/sites/3/2016/04/gp100_block_diagram-1.png)"

### CPU chip layout

![Intel CPU die shot](https://cdn.arstechnica.net/wp-content/uploads/2011/11/core_i7_lga_2011_die-4ec188e-intro.jpg)

In [4]:
[CUDA.capability(dev) for dev in CUDA.devices()]

1-element Vector{VersionNumber}:
 v"6.0.0"

## Calling GPU on arrays

In [5]:
N = 10^6

1000000

In [6]:
begin
	x_d = CUDA.fill(1.0f0, N)  # a vector stored on the GPU filled with 1.0 (Float32)
	y_d = CUDA.fill(2.0f0, N)  # a vector stored on the GPU filled with 2.0
end;  # Suppress output to notebook, unless enable scalar indexing

In [7]:
begin
	y_d .+= x_d
	@test all(Array(y_d) .== 3.0f0)
end

Test Passed

In [8]:
function add_broadcast!(y, x)
    CUDA.@sync y .+= x
    return
end

add_broadcast! (generic function with 1 method)

In [9]:
	x_h = fill(1.0f0,N)
	y_h = fill(2.0f0,N)
	@time y_h .+= x_h
	@time y_h .+= x_h
	@time CUDA.@sync y_h .+= x_h
	@time CUDA.@sync y_h .+= x_h;

  0.061591 seconds (90.27 k allocations: 6.043 MiB)
  0.000571 seconds (2 allocations: 64 bytes)
  0.000577 seconds (2 allocations: 64 bytes)
  0.000645 seconds (2 allocations: 64 bytes)


In [10]:
x_d, add_broadcast!
z_d = CUDA.fill(2.0f0, N)  
@time  add_broadcast!(z_d, x_d)
@time add_broadcast!(z_d, x_d)
@time CUDA.@sync add_broadcast!(z_d, x_d)
@time CUDA.@sync add_broadcast!(z_d, x_d)

  0.028065 seconds (12.65 k allocations: 892.506 KiB)
  0.000070 seconds (31 allocations: 1.859 KiB)
  0.000070 seconds (31 allocations: 1.859 KiB)
  0.000074 seconds (31 allocations: 1.859 KiB)


## How to use GPUs very ineefficiently

In [11]:
function gpu_add1!(y, x)
    for i = 1:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

gpu_add1! (generic function with 1 method)

In [12]:
function bench_gpu1!(y, x)
    CUDA.@sync begin
        @cuda gpu_add1!(y, x)
    end
end

bench_gpu1! (generic function with 1 method)

In [13]:
begin
	bench_gpu1!(y_d, x_d)
	@elapsed bench_gpu1!(y_d, x_d)
end

0.178713866

## Parallelize over multiple threads

**Q:** Can you run 'for' loops in parallel on a GPU?


In [14]:
function gpu_add2!(y, x)
    index = threadIdx().x    # this example only requires linear indexing, so just use `x`
    stride = blockDim().x
    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

gpu_add2! (generic function with 1 method)

In [15]:
let
	fill!(y_d, 2)
	@cuda threads=256 gpu_add2!(y_d, x_d)
	@test all(Array(y_d) .== 3.0f0)
end

Test Passed

In [16]:
function bench_gpu2!(y, x)
    CUDA.@sync begin
        @cuda threads=256 gpu_add2!(y, x)
    end
end


bench_gpu2! (generic function with 1 method)

In [17]:
begin
	bench_gpu2!(y_d, x_d)
	@elapsed bench_gpu2!(y_d, x_d)
end

0.002026105

## Paralelizing with multiple blocks
![CUDA array indexing](https://cuda.juliagpu.org/dev/tutorials/intro1.png)

In [18]:
function gpu_add3!(y, x)
    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if index <= length(y)
		@inbounds y[index] += x[index]
	end
	return
end

gpu_add3! (generic function with 1 method)

In [19]:
let
	numblocks = ceil(Int, N/256)
	
	fill!(y_d, 2)
	@cuda threads=256 blocks=numblocks gpu_add3!(y_d, x_d)
	@test all(Array(y_d) .== 3.0f0)
end

Test Passed

In [20]:
function bench_gpu3!(y, x)
    numblocks = ceil(Int, length(y)/256)
    CUDA.@sync begin
        @cuda threads=256 blocks=numblocks gpu_add3!(y, x)
    end
end

bench_gpu3! (generic function with 1 method)

In [21]:
begin
	bench_gpu3!(y_d, x_d)
	@elapsed bench_gpu3!(y_d, x_d)
end

5.2046e-5

### Choosing Number of threads/block & Number of blocks

In [22]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MULTIPROCESSOR_COUNT)

56

In [23]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)

1024

In [24]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_SHARED_MEMORY_PER_BLOCK)/1024 #KB

48.0

In [25]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_WARP_SIZE)

32

In [26]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_BLOCK_DIM_X), 
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_BLOCK_DIM_Y),
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_BLOCK_DIM_Z)

(1024, 1024, 64)

In [27]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_GRID_DIM_X), 
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_GRID_DIM_Y), CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_GRID_DIM_Z)

(2147483647, 65535, 65535)

In [28]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_L2_CACHE_SIZE)/1024^2 # MB

3.0

In [29]:
begin
	kernel = @cuda launch=false gpu_add3!(y_d, x_d)
	config = launch_configuration(kernel.fun)
	threads3 = min(N, config.threads)
	blocks3 = cld(N, threads3)
	threads3, blocks3
end

(1024, 977)

In [30]:
begin
	fill!(y_d, 2)
	kernel(y_d, x_d; threads=threads3, blocks=blocks3)
	@test all(Array(y_d) .== 3.0f0)
end

Test Passed

In [31]:
function bench_gpu4!(y, x)
    kernel = @cuda launch=false gpu_add3!(y, x)
    config = launch_configuration(kernel.fun)
    threads4 = min(length(y), config.threads)
    blocks4 = cld(length(y), threads4)

    CUDA.@sync begin
        kernel(y, x; threads=threads4, blocks=blocks4)
    end
end

bench_gpu4! (generic function with 1 method)

In [32]:
begin
	bench_gpu4!(y_d, x_d)
	@elapsed bench_gpu4!(y_d, x_d)
end

5.0521e-5

### Generating Pseudo-random numbers on the GPU

In [33]:
@elapsed rand_nums_h = rand(N)

0.110167669

In [34]:
@elapsed (CUDA.@sync rand_nums_d = CUDA.rand(N))

0.567231846

In [35]:
rand_nums_h

1000000-element Vector{Float64}:
 0.07158018159398527
 0.12150290415310305
 0.2806580697314387
 0.7576182492403284
 0.4481729125409478
 0.9180841377375324
 0.1958425617052878
 0.7439165099824575
 0.5836985731067483
 0.01361140458826493
 0.17111914127773653
 0.36088205044028
 0.05089161598456837
 ⋮
 0.24148329497020182
 0.9360944261791848
 0.5740321860086848
 0.40953835123957405
 0.3389203108371428
 0.7139580075109151
 0.5551717300540272
 0.7324362035427616
 0.9129771162578413
 0.9383278952428211
 0.7922551508170057
 0.5280522187457685

In [36]:
collect(rand_nums_d)

1000000-element Vector{Float32}:
 0.511338
 0.9368336
 0.6444489
 0.8800617
 0.7683473
 0.01727279
 0.8722887
 0.79843414
 0.06122761
 0.78966343
 0.4086514
 0.23382325
 0.15878306
 ⋮
 0.43647316
 0.8613937
 0.36172053
 0.48447618
 0.30490714
 0.30156836
 0.2639223
 0.3211724
 0.36164367
 0.87946635
 0.90603137
 0.34840906

In [37]:
CUDA.@allowscalar rand_nums_d[1]

0.511338f0

## Using GPU for a more substantial function

In [51]:
"""
   ecc_anom_init_guess_danby(mean_anomaly, eccentricity)
Returns initial guess for the eccentric anomaly for use by itterative solvers of Kepler's equation for bound orbits.  
Based on "The Solution of Kepler's Equations - Part Three"
Danby, J. M. A. (1987) Journal: Celestial Mechanics, Volume 40, Issue 3-4, pp. 303-312 (1987CeMec..40..303D)
"""
function ecc_anom_init_guess_danby(M::Real, ecc::Real)
	my_pi = convert(typeof(M),pi)
    @assert -2*my_pi<= M <= 2*my_pi
	@assert 0 <= ecc <= 1.0
    k = convert(typeof(M), 0.85)
    if  M < zero(M)
		M += 2*my_pi
	end
    E = (M<my_pi) ? M + k*ecc : M - k*ecc
end;

In [52]:
"""
   update_ecc_anom_laguerre(eccentric_anomaly_guess, mean_anomaly, eccentricity)
Update the current guess for solution to Kepler's equation
  
Based on "An Improved Algorithm due to Laguerre for the Solution of Kepler's Equation"
   Conway, B. A.  (1986) Celestial Mechanics, Volume 39, Issue 2, pp.199-211 (1986CeMec..39..199C)
"""
function update_ecc_anom_laguerre(E::Real, M::Real, ecc::Real)
  es = ecc*sin(E)
  ec = ecc*cos(E)
  #(es, ec) = ecc .* sincos(E)  # Does combining them provide any speed benefit?
  F = (E-es)-M
  Fp = one(M)-ec
  Fpp = es
  n = 5
  root = sqrt(abs((n-1)*((n-1)*Fp*Fp-n*F*Fpp)))
  denom = Fp>zero(E) ? Fp+root : Fp-root
  return E-n*F/denom
end;

In [53]:
 function calc_ecc_anom(mean_anom::Real, ecc::Real, tol::Real = 1.0e-8)
  	@assert 0 <= ecc <= 1.0
	@assert 1e-16 <= tol < 1
  	#M = rem2pi(mean_anom,RoundNearest)
    M = mod(mean_anom,2*convert(typeof(mean_anom),pi))
    E = ecc_anom_init_guess_danby(M,ecc)
	local E_old
    max_its_laguerre = 200
    for i in 1:max_its_laguerre
       E_old = E
       E = update_ecc_anom_laguerre(E_old, M, ecc)
       if abs(E-E_old) < tol break end
    end
    return E
end

calc_ecc_anom (generic function with 2 methods)

In [54]:
function calc_true_anom(ecc_anom::Real, e::Real)
	true_anom = 2*atan(sqrt((1+e)/(1-e))*tan(ecc_anom/2))
end

calc_true_anom (generic function with 1 method)

In [55]:
function gpu_calc_ecc_anom!(ecc_anom, mean_anom, ecc, tol)
	#@assert 1e-16 <= tol < 1
    #@assert size(ecc_anom) == size(mean_anom) == size(ecc) 
	index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
	if index <= length(ecc_anom)
		@inbounds ecc_anom[index] = calc_ecc_anom(mean_anom[index],ecc[index])
	end
	return 
end

gpu_calc_ecc_anom! (generic function with 1 method)

In [56]:
function bench_gpu_calc_ecc_anom!(ecc_anom, mean_anom, ecc, tol)
    @assert size(ecc_anom) == size(mean_anom) == size(ecc) 
	kernel = @cuda launch=false gpu_calc_ecc_anom!(ecc_anom, mean_anom, ecc, tol)
    config = launch_configuration(kernel.fun)
    threads = min(length(ecc_anom), config.threads)
    blocks = cld(length(ecc_anom), threads)

    CUDA.@sync begin
        kernel(ecc_anom, mean_anom, ecc, tol; threads=threads, blocks=blocks)
    end
end

bench_gpu_calc_ecc_anom! (generic function with 1 method)

In [57]:
M=1000

1000

In [58]:
begin
	ecc_anom_d = CUDA.zeros(Float64,M)
	ecc_d = CUDA.rand(Float64,M)
	mean_anom_d = CUDA.rand(Float64,M)
	CUDA.@sync mean_anom_d .*= 2π
	tol = 1e-8
	kepler_kernel = @cuda launch=false gpu_calc_ecc_anom!(ecc_anom_d, mean_anom_d,ecc_d,tol)
	kepler_config = launch_configuration(kepler_kernel.fun)
end

(blocks = 112, threads = 576)

In [59]:
1000*@elapsed bench_gpu_calc_ecc_anom!(ecc_anom_d,mean_anom_d,ecc_d,tol)

61.09407

In [60]:
collect(ecc_anom_d)

1000-element Vector{Float64}:
 5.34313912333136
 3.6755896634842444
 2.154877383862004
 1.2888247317686994
 3.442135039653894
 4.875556961720868
 3.1113295603189877
 2.98720570909686
 1.8957695357397137
 4.455569247540566
 0.6476035138644487
 5.471617118509137
 2.6082256579598577
 ⋮
 2.8829294233705043
 5.283121956751919
 5.880804327374942
 2.440126331531619
 4.767433445898621
 5.350860151603896
 2.6417675371436373
 5.526644572489476
 6.193152852348971
 2.9505108998937435
 0.36280589923210554
 4.124672336556435

In [61]:
begin
	ecc_anom_h = collect(ecc_anom_d)
	mean_anom_h = collect(mean_anom_d)
	ecc_h = collect(ecc_d)
	
	1000*@elapsed ecc_anom_h_comp = calc_ecc_anom.(mean_anom_h,ecc_h)
end

129.14849

In [62]:
maximum(abs.(ecc_anom_h_comp .- ecc_anom_h))

1.7763568394002505e-15

## GPU Reductions

![GPU Reduction](gpu_reduction.png)

### Q&A

**Q:** Is it always faster to use GPU than CPU? When is CPU a better option than GPU?
**A:** No.  See Lab 8.

**Q:** It seems like GPUs can be quite complicated to work with, and in some cases favor lower precision outputs for higher performance. In what circumstances would one choose to work on a GPU rather than CPUs?
**A:** When there's a >10x performance benefit (for your required accuracy). 

**Q:** I am still unsure about the implementation of a GPU kernal. How common is this and how should we know whether to use it over a generic GPU array interface?
**A:** Generaly, I'd try the array interface first (if you can).  Then decide if you want to try to get further performance boost by reducing memory transfers.

**Q:** Can you explain what does it mean that "all calls to the GPU are scheduled asynchronous"?
**A:** You have to exp

**Q:** How can we determine whether a certain Julia type is compatible with a GPU?        
**A:** Your standard integers, floats and complex types are implemented.  Arrays, and that's about it.

**Q:** Why is there latency for RAM to communicate with the GPU's VRAM? Could a system be converted fully to VRAM or can VRAM not operate as RAM?    
**A:** They're two separate memory systems.
